In [1]:
# Get the grain id we care about
def get_actual_grain_id(grain_id):
    return grain_id % 10000

# Get a map from grain id to phi1, Phi, phi2
def get_grain_id_to_phis(file):
    grain_id_to_phis = {}
    with open(file, 'r') as r:
        r.readline()
        for line in r:
            tokens = line.split()
            grain_id = get_actual_grain_id(int(tokens[0]))
            phi1 = float(tokens[30])
            Phi = float(tokens[32])
            phi2 = float(tokens[34])
            grain_id_to_phis[grain_id] = (phi1, Phi, phi2)
    return grain_id_to_phis

In [2]:
# misorientation
from numpy import *

# Citation: Dr. Ashley Spear, University of Utah
def get_disorientation_1(phi1_1, Phi_1, phi2_1, phi1_2, Phi_2, phi2_2):

    # uses quaternions

    # these should be in radians
    phi1 = [phi1_1, phi1_2]
    phi = [Phi_1, Phi_2]
    phi2 = [phi2_1, phi2_2]

    Q = {}
    for i in range(2):
        s = sin(0.5 * phi[i])
        c = cos(0.5 * phi[i])
        s1 = sin(0.5 * (phi1[i]-phi2[i]))
        c1 = cos(0.5 * (phi1[i]-phi2[i]))
        s2 = sin(0.5 * (phi1[i]+phi2[i]))
        c2 = cos(0.5 * (phi1[i]+phi2[i]))

        q1 = s*c1
        q2 = s*s1
        q3 = c*s2
        q4 = c*c2

        Q[i] = [q1, q2, q3, q4]

    qresult = [0, 0, 0, 0]
    qresult[0] = abs(Q[0][0]*Q[1][3]-Q[0][3]*Q[1][0]+Q[0][1]*Q[1][2]-Q[0][2]*Q[1][1])
    qresult[1] = abs(Q[0][1]*Q[1][3]-Q[0][3]*Q[1][1]+Q[0][2]*Q[1][0]-Q[0][0]*Q[1][2])
    qresult[2] = abs(Q[0][2]*Q[1][3]-Q[0][3]*Q[1][2]+Q[0][0]*Q[1][1]-Q[0][1]*Q[1][0])
    qresult[3] = abs(Q[0][3]*Q[1][3]+Q[0][0]*Q[1][0]+Q[0][1]*Q[1][1]+Q[0][2]*Q[1][2])

    temp = sort(qresult)

    maximum = temp[3]
    maximum2 = temp[2]

    disor = maximum
    if maximum < (maximum+maximum2)/sqrt(2.0):
        disor = (maximum+maximum2)/sqrt(2.0)

    rtmp = sum(qresult)/2.0
    if disor < rtmp:
        disor = rtmp

    if disor > 1:
        disor = 1.0

    if disor < -1:
        disor = -1.0

    return arccos(disor)*360/pi

In [4]:
# Get the grain ids mapped to phis
grain_id_to_phis = get_grain_id_to_phis('../../data/0-grain-ids-phis.txt')

natee = 0

# Read from the distance vector file
with open('../../data/3-distance-vector.csv', 'r') as r:
    # Write to the phis/misorientation file
    with open('../../data/4-phis-misorientation.csv', 'w') as w:
        # Write the file headers
        headers = ('dadN,crack_id,theta,x,y,z,on_crack_front,grain_id,nearest_grain_boundary_x,nearest_grain_boundary_y,'
                   'nearest_grain_boundary_z,nearest_grain_boundary_id,distance_to_grain_boundary,'
                   'vector_to_grain_boundary_x,vector_to_grain_boundary_y,vector_to_grain_boundary_z,'
                   'magnitude_of_vector_to_grain_boundary,unit_vector_to_grain_boundary_x,unit_vector_to_grain_boundary_y,unit_vector_to_grain_boundary_z,phi1,Phi,phi2,'
                   'nearest_grain_phi1,nearest_grain_Phi1,nearest_grain_phi2,misorientation\n')
        w.write(headers)
        
        # Skip the headers of the file
        r.readline()
        
        for line in r:
            # Read in the tokens
            tokens_ = line.split(',')
            tokens = []
            # Get rid of whitespace
            for token in tokens_:
                tokens.append(token.strip())
            
            dadN = tokens[0]
            crack_id = tokens[1]
            theta = tokens[2]
            x = tokens[3]
            y = tokens[4]
            z = tokens[5]
            on_crack_front = tokens[6] == 'True'
            grain_id = get_actual_grain_id(int(tokens[7]))
            nearest_grain_boundary_x = tokens[8]
            nearest_grain_boundary_y = tokens[9]
            nearest_grain_boundary_z = tokens[10]
            nearest_grain_boundary_id = get_actual_grain_id(int(tokens[11]))
            distance_to_grain_boundary = tokens[12]
            vector_to_grain_boundary_x = tokens[13]
            vector_to_grain_boundary_y = tokens[14]
            vector_to_grain_boundary_z = tokens[15]
            magnitude_of_vector_to_grain_boundary = tokens[16]
            unit_vector_to_grain_boundary_x = tokens[17]
            unit_vector_to_grain_boundary_y = tokens[18]
            unit_vector_to_grain_boundary_z = tokens[19]

            # Get the phis for the current grain id
            phi1, Phi, phi2 = grain_id_to_phis[grain_id]
            
            # Find the phis for the nearest grain boundary's id
            nearest_grain_phi1, nearest_grain_Phi, nearest_grain_phi2 = grain_id_to_phis[nearest_grain_boundary_id]
            
            # Get the misorientation between the two grains
            misorientation = get_disorientation_1(phi1,Phi,phi2,nearest_grain_phi1,nearest_grain_Phi,nearest_grain_phi2)

            # Write it to the file
            to_write = '{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n'\
                                .format(dadN,crack_id,theta,x,y,z,on_crack_front,
                                grain_id,nearest_grain_boundary_x,nearest_grain_boundary_y,
                                nearest_grain_boundary_z,nearest_grain_boundary_id,distance_to_grain_boundary,
                                vector_to_grain_boundary_x,vector_to_grain_boundary_y,vector_to_grain_boundary_z,
                                magnitude_of_vector_to_grain_boundary,unit_vector_to_grain_boundary_x,
                                unit_vector_to_grain_boundary_y, unit_vector_to_grain_boundary_z, phi1, Phi, phi2, 
                                nearest_grain_phi1, nearest_grain_Phi, nearest_grain_phi2, misorientation)
            w.write(to_write)